# Climate-Informed Cybersecurity Preparedness

This notebook demonstrates how to use NOAA Climate Data Products to identify areas of high risk for cybersecurity threats related to natural disasters like Hurricane Helene in North Carolina, focusing on Asheville and Greenville.

**Objectives:**
- **Identify Critical Infrastructure**: Locate data centers throughout North Carolina with the aid of publicly-available websites and web scraping tools.
- **Determine High Risk Zones**: Assess areas expoded to acute precipitation amounts and identify data centers most vulnerable to flooding and damage.


> ### Disclaimer: Understanding the Complexity of Hydrology
> 
> Hydrology, the branch of science dedicated to understanding water movement and distribution, involves a range of factors that influence flood risk and water resource management. Additional factors may include soil saturation, watershed characteristics, river flow dynamics, evaporation rates, and topography. While this example uses precipitation totals as an indicator, it is important to note that **precipitation totals alone are not sufficient for a comprehensive climate-adjacent risk assessment**.
> 
> 
> This example represents a simplified approach to climate-related risk assessment. Students are encouraged to build on this foundation by adding more complex variables from various datasets. Explore more NOAA datasets [here](https://www.arcgis.com/home/search.html?restrict=false&sortField=relevance&sortOrder=desc&searchTerm=owner%3A%22NWS.NCEP.CPC_noaa%22#content).




## Case Study

In the face of increasing flooding volatility due to climate change, you are apart of a technology firm that is proactively assessing potential risks to data centers across its North Carolina footprint, especially in light of unprecedented damage brought by Hurricane Helene. The firm needs to conduct a preliminary analysis to evaluate the need for redundant data centers in safer locations. As part of this assessment, we need to identifying potential weaknesses in our Asheville data center's network infrastructure, which might be at greater risk for cyber attacks if physical damage from flooding were to occur again. Our goal is to develop a robust strategy to strengthen our resilience against climate-related cyber threats and ensure the continuity of our operations.

In [1]:
# Let's install all python packages we need.
!pip install geopandas pandas matplotlib shapely contextily branca

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   --------------------- ------------------ 0.8/1.4 MB 4.8 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 5.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/16.2 MB ? eta -:--:--
   --- ------------------------------------ 1.6/16.2 MB 7.6 MB/s eta 0:00:02
   -------- ------------------------------- 3.4/16.2 MB 9.2 MB/s eta 0:00:02
   ---------------- ----------------------- 6.8/16.2 MB 10.8 MB/s eta 0:00:01
   ------------------------------ --------- 12.3/16.2 MB 14.6 MB/s eta 0:00:01
   ---------------------------------------- 16.2/16.2 MB 16.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.2 MB ? eta -:--:--
   -------------------------------------- - 6.0/6.2 MB 30.7 MB/s eta 0:00:01
   ---------------------------------------- 6.2/6.2 MB 27.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.5 MB ? eta -:--:--
   ----------- -----------

In [2]:
# Import necessary libraries
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import contextily as ctx

## Data Center Locations: Generating the CSV File

For this example, I generated a CSV file of North Carolina data center locations to simulate a fictitious infrastructure footprint. I used the ParseHub app, which is a tool for web scraping, to extract data from the following website:

[Data Centers in North Carolina - DataCenters.com](https://www.datacenters.com/locations/united-states/north-carolina)

[ParseHub](https://www.parsehub.com/quickstart) allowed us to automate the process of collecting relevant information, such as the names and addresses of data centers listed on the website. I then exported this data into a CSV format. This dataset will serve as the foundation for our analysis, where we'll explore the cybersecurity preparedness of critical infrastructure in North Carolina.



In [3]:
# Read Data Center Locations from the CSV File

# Replace 'data_centers.csv' with the path to your CSV file
infra_df = pd.read_csv('data_centers.csv')

# Display the DataFrame
print("Data Centers from CSV:")
infra_df


Data Centers from CSV:


,datacenter_name_name,datacenter_name_address,datacenter_name_address_url,latitude,longitude
0,@Charlotte Data Center Campus,"131 Riverside Ct, Kings Mountain, NC, USA",https://www.datacenters.com/t5-charlotte-campus,35.2397,-81.3437
1,Raleigh Data Center,"5150 McCrimmon Parkway #423, Morrisville, NC 2...",https://www.datacenters.com/flexential-raleigh,35.8615,-78.8263
2,DUR1 Raleigh-Durham Data Center,"2223 Northeast Creek Parkway, Durham, NC, USA",https://www.datacenters.com/cyrusone-dur1-rale...,35.9325,-78.8453
3,Raleigh Data Center,"5301 Departure Drive, Raleigh, NC, USA",https://www.datacenters.com/tierpoint-raleigh,35.8618,-78.6003
4,Charlotte Data Center,"5821 Fairview Rd, Charlotte, NC, USA",https://www.datacenters.com/charlotte-colocati...,35.1526,-80.8337
5,Raleigh - RTP Data Center,"99 TW Alexander Drive, Durham, NC 27709, USA",https://www.datacenters.com/tierpoint-raleigh-rtp,35.9188,-78.8973
6,Charlotte - North Data Center,"10105 David Taylor Drive, Charlotte, NC, USA",https://www.datacenters.com/flexential-charlot...,35.3347,-80.7805
7,Raleigh 1 Data Center,"5301 Departure Drive, Raleigh, NC, USA",https://www.datacenters.com/lumen-raleigh-1,35.8618,-78.6003
8,Charlotte CLT10 Data Center,"113 North Myers Street, Charlotte, NC, USA",https://www.datacenters.com/digital-realty-cha...,35.2175,-80.8312
9,Asheville Data Center,"100 Technology Dr ste c, Asheville, NC, USA",https://www.datacenters.com/dartpoints-asheville,35.6168,-82.5642


## Visualizing Data Centers and GEFS Precipitation Forecast on a Map

In this map, we will display the locations of data centers across North Carolina alongside the **Global Ensemble Forecast System (GEFS)** Day 12 precipitation forecast. The GEFS dataset, produced by the National Oceanic and Atmospheric Administration (NOAA), is a global weather forecast model that combines multiple weather simulations (or “ensemble members”) to predict conditions over a 16-day period.

The **Day 12 forecast** specifically provides precipitation predictions 12 days into the future. This forecast is valuable for assessing potential rainfall, especially during extreme weather events, allowing organizations to plan and respond to anticipated conditions. By overlaying this data with data center locations, we can evaluate areas at risk from forecasted precipitation levels.


In [5]:
import folium



# Define the center point for the map (Asheville, NC)
map_center = [35.5951, -82.5515]

# Create a Folium map centered on Asheville, using CartoDB Positron as a base layer
m = folium.Map(location=map_center, zoom_start=8, tiles="CartoDB positron", name="United States Boundaries")

# Add the ArcGIS MapServer as a tile layer. You can copy and paste the two lines of code below 
# and subsitute the url with anyweb map server endpoint of your choice. [Look here to find more datasets](https://www.arcgis.com/home/search.html?restrict=false&sortField=relevance&sortOrder=desc&searchTerm=owner%3A%22NWS.NCEP.CPC_noaa%22#content)
arcgis_tile_url = "https://tiles.arcgis.com/tiles/C8EMgrsFcRFL6LrL/arcgis/rest/services/prcp_gefs_day12_thresh_2/MapServer/tile/{z}/{y}/{x}"
folium.TileLayer(arcgis_tile_url, name="Precipitation Totals", attr="ArcGIS").add_to(m)

arcgis_tile_url = "https://tiles.arcgis.com/tiles/C8EMgrsFcRFL6LrL/arcgis/rest/services/nrmSeasTemp/MapServer/tile/{z}/{y}/{x}"
folium.TileLayer(arcgis_tile_url, name="Precipitation Totals", attr="ArcGIS").add_to(m)


# Add each data center as a marker on the map
for _, row in infra_df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['datacenter_name_name'],
        icon=folium.Icon(color="blue", icon="cloud")
    ).add_to(m)

# Display the map with infrastructure and precipitation data
m


The 20%, 40%, 60% values in the GEFS Day 12 precipitation forecast represent probability contours for receiving a certain amount of precipitation. For this demostration, we chose to look at areas recieving more than 2 inches of rainfall. These contours indicate the likelihood, based on the ensemble of forecast models, that a given area will experience precipitation above a specified threshold of 2 inches.

Here's a breakdown:

- 20% Contour: There’s a 20% chance that this area will receive at least the threshold amount of precipitation (e.g., 2 inches) within the forecast period.
- 40% Contour: A 40% chance of meeting or exceeding the precipitation threshold.
- 60% Contour (and higher): Higher probabilities, indicating areas with a greater likelihood of substantial rainfall.


## Conclusion

As of October 2024, there is currently no predicted rainfall greater than 2 inches for the North Carolina region. Instead, all significant rainfall predictions are concentrated around the tail of Central America.

While this may be the case now, it’s important to remember that the GEFS model updates regularly, providing new forecasts as weather conditions evolve. This notebook serves as a quick and easily executable tool, allowing for rapid assessment of forecasted precipitation alongside critical infrastructure locations when locally volatile weather is anticipated.


## Contact
- Kenneth Witt: kwitt@wvhtf.org
- Lauryn Eldridge: leldridge@wvhtf.org